# Initialize variables

In [13]:
tag_list = ['paypal','stripe-payments']
fetch_start_date = '2019-01-01'
fetch_end_date = '2019-01-01'

# Setup working directory and date range

In [6]:
import os
current_dir = os.getcwd()

In [16]:
import pandas as pd
pd.set_option('display.max_columns', 999)

In [9]:
from datetime import datetime
today_date = datetime.today().strftime('%Y-%m-%d')
start_date= pd.date_range(fetch_start_date,today_date , freq='MS').strftime('%Y-%m-%d') # first of the month
end_date  = pd.date_range(fetch_end_date,today_date , freq='M').strftime('%Y-%m-%d')  # last of the month
N = len(end_date) # not start_date, which has one additional month
print(f'start_date:{start_date}')
print(f'end_date:{end_date}')

start_date:Index(['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01', '2019-05-01',
       '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01',
       '2019-11-01', '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01',
       '2020-04-01'],
      dtype='object')
end_date:Index(['2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31',
       '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
       '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31'],
      dtype='object')


# Get data from Stackoverflow

I had trouble getting my SSL certificate on python/anaconda to work. So I disabled it in the StackAPI package and disabled the warnings for now

https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings

I get warning because in stackapi.py, line 183, I added verify=False
C:\Users\mkommaraju\AppData\Local\Continuum\anaconda3\lib\site-packages\stackapi\stackapi.py

StackAPI documentation https://stackapi.readthedocs.io/en/latest/

In [10]:
import urllib3
urllib3.disable_warnings()

In [11]:
from stackapi import StackAPI
SITE = StackAPI('stackoverflow')

In [14]:
questions_items_full_list=[]
for tag in tag_list:
    for i in range(N):
        print("'{}' tag, start={}, end={}".format(tag, start_date[i], end_date[i]) )
        questions = SITE.fetch('questions', 
                               fromdate=datetime.strptime(start_date[i], '%Y-%m-%d'), 
                               todate=datetime.strptime(end_date[i], '%Y-%m-%d'), 
                               tagged=tag)
        questions_items = questions.get('items')
        questions_items[1]
        for q in questions_items:
            q.update({'search_tag':tag})
        questions_items_full_list = questions_items_full_list + questions_items

df = pd.DataFrame(questions_items_full_list)
df["creation_date_str"] = [datetime.fromtimestamp(t).strftime('%Y-%m-%d') for t in df.creation_date]
df["is_answered_flag"] = [1 if f==True else 0 for f in df.is_answered]

'paypal' tag, start=2019-01-01, end=2019-01-31
'paypal' tag, start=2019-02-01, end=2019-02-28
'paypal' tag, start=2019-03-01, end=2019-03-31
'paypal' tag, start=2019-04-01, end=2019-04-30
'paypal' tag, start=2019-05-01, end=2019-05-31
'paypal' tag, start=2019-06-01, end=2019-06-30
'paypal' tag, start=2019-07-01, end=2019-07-31
'paypal' tag, start=2019-08-01, end=2019-08-31
'paypal' tag, start=2019-09-01, end=2019-09-30
'paypal' tag, start=2019-10-01, end=2019-10-31
'paypal' tag, start=2019-11-01, end=2019-11-30
'paypal' tag, start=2019-12-01, end=2019-12-31
'paypal' tag, start=2020-01-01, end=2020-01-31
'paypal' tag, start=2020-02-01, end=2020-02-29
'paypal' tag, start=2020-03-01, end=2020-03-31
'stripe-payments' tag, start=2019-01-01, end=2019-01-31
'stripe-payments' tag, start=2019-02-01, end=2019-02-28
'stripe-payments' tag, start=2019-03-01, end=2019-03-31
'stripe-payments' tag, start=2019-04-01, end=2019-04-30
'stripe-payments' tag, start=2019-05-01, end=2019-05-31
'stripe-payment

In [17]:
df.head()

,tags,owner,is_answered,view_count,accepted_answer_id,answer_count,score,last_activity_date,creation_date,question_id,link,title,search_tag,last_edit_date,migrated_from,protected_date,closed_date,closed_reason,bounty_amount,bounty_closes_date,creation_date_str,is_answered_flag
0,"[paypal, paypal-sandbox, paypal-subscriptions]","{'reputation': 4604, 'user_id': 1319542, 'user...",True,606,54165510.0,1,0,1577543072,1547219141,54149111,https://stackoverflow.com/questions/54149111/w...,What is the difference between Paypal &quot;pr...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-11,1
1,"[spring-boot, paypal, access-token, resttempla...","{'reputation': 167, 'user_id': 9673158, 'user_...",False,260,NaN,1,0,1577362298,1548220554,54320426,https://stackoverflow.com/questions/54320426/h...,How to set paypal get access token param of PO...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-22,0
2,"[paypal, paypal-rest-sdk]","{'reputation': 1361, 'user_id': 4497941, 'user...",False,39,NaN,1,1,1565672506,1548502125,54377965,https://stackoverflow.com/questions/54377965/p...,PayPal Payout API limits?,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-26,0
3,"[paypal, applepay, google-pay, venmo, samsung-...","{'reputation': 71, 'user_id': 9072072, 'user_t...",True,147,NaN,1,2,1563018676,1547026474,54106958,https://stackoverflow.com/questions/54106958/i...,Implementing Peer to Peer payments,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-09,1
4,"[javascript, kotlin, paypal, downshift]","{'reputation': 31, 'user_id': 5098212, 'user_t...",False,152,NaN,1,3,1561444720,1548566276,54385291,https://stackoverflow.com/questions/54385291/c...,Creating a Kotlin React Wrapper around Paypal ...,paypal,1.548568e+09,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-26,0


In [23]:
df.title[0]

'What is the difference between Paypal &quot;preapproved payments&quot; and &quot;automatic payments&quot;?'

The 'title' column has special characcters, convert to a claner format to improve readability

In [24]:
import html.parser; 
html.parser.HTMLParser().unescape(df.title[0])

C:\Users\mkommaraju\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  


'What is the difference between Paypal "preapproved payments" and "automatic payments"?'

In [25]:
html.unescape(df.title[0])

'What is the difference between Paypal "preapproved payments" and "automatic payments"?'

In [26]:
df["title_clean"] = [html.parser.HTMLParser().unescape(f) for f in df.title]

C:\Users\mkommaraju\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """Entry point for launching an IPython kernel.


In [27]:
df.columns

Index(['tags', 'owner', 'is_answered', 'view_count', 'accepted_answer_id',
       'answer_count', 'score', 'last_activity_date', 'creation_date',
       'question_id', 'link', 'title', 'search_tag', 'last_edit_date',
       'migrated_from', 'protected_date', 'closed_date', 'closed_reason',
       'bounty_amount', 'bounty_closes_date', 'creation_date_str',
       'is_answered_flag', 'title_clean'],
      dtype='object')

# Save data to csv file

In [28]:
df.to_csv('stackoverflow'+today_date+'.csv',index=False)

# Search the questions on Google and get results for analysis

In [29]:
from urllib.parse import urlparse
from googlesearch import search 
# to search 
#query = df.search_tag[0] + ' ' + df.title_clean[0]
#query = 'paypal What is the difference between Paypal preapproved payments and automatic payments'

In [30]:
question_id_list = []
result_url_list = []
result_url_short_list = []
search_tag_list = []

N_questions = 5
#N_questions = df.shape[0]
for i in range(N_questions):
    query = df.search_tag[i] + ' ' + df.title_clean[i] # search for tag+question
    search_result = search(query, tld="com", num=10, stop=10, pause=1)
    search_result_list = list(search_result)
    short_result_url = []
    print(i)
    for res in search_result_list: 
        search_tag_list.append(df.search_tag[i])
        question_id_list.append(df.question_id[i])
        result_url_list.append(res)
        result_url_short_list.append(urlparse(res).netloc)

df_google_search = pd.DataFrame({'search_tag':search_tag_list,
                                 'question_id':question_id_list,
                                 'result_url':result_url_list,
                                 'result_url_short':result_url_short_list
                                 })
df_google_search.reset_index(drop=False, inplace=True)
df_google_search['search_rank'] = df_google_search.groupby(['search_tag','question_id'])['index'].rank(method='first')

0
1
2
3
4


In [32]:
df_google_search.head()

,index,search_tag,question_id,result_url,result_url_short,search_rank
0,0,paypal,54149111,https://stackoverflow.com/questions/54149111/w...,stackoverflow.com,1.0
1,1,paypal,54149111,https://www.paypal.com/us/webapps/mpp/popup/ab...,www.paypal.com,2.0
2,2,paypal,54149111,https://www.groovypost.com/howto/switch-paypal...,www.groovypost.com,3.0
3,3,paypal,54149111,https://www.mymoneyblog.com/preapproved-paymen...,www.mymoneyblog.com,4.0
4,4,paypal,54149111,https://www.paypal-community.com/t5/Sending-mo...,www.paypal-community.com,5.0


In [33]:
df_google_search.to_csv('df_google_search'+today_date+'.csv',index=False)

# Combine the stackoverflow results with the Google Search results

In [34]:
df = pd.read_csv('stackoverflow'+today_date+'.csv')
df.columns
df.tags
df_tags_list = []
df_question_id_list = []
for i in range(df.shape[0]):
    df_tags = df.tags[i].strip('[]').replace(' ','').replace('\'','').split(',')
    q = df.question_id[i]
    for t in df_tags:
        df_tags_list.append(t)
        df_question_id_list.append(q)
df_tags_map = pd.DataFrame({'question_id':df_question_id_list,
                            'tag_id':df_tags_list
                            })
df_tags_questions = df_tags_map.merge(df, left_on='question_id', right_on='question_id')

In [35]:
df_tags_questions.head()

,question_id,tag_id,tags,owner,is_answered,view_count,accepted_answer_id,answer_count,score,last_activity_date,creation_date,link,title,search_tag,last_edit_date,migrated_from,protected_date,closed_date,closed_reason,bounty_amount,bounty_closes_date,creation_date_str,is_answered_flag,title_clean
0,54149111,paypal,"['paypal', 'paypal-sandbox', 'paypal-subscript...","{'reputation': 4604, 'user_id': 1319542, 'user...",True,606,54165510.0,1,0,1577543072,1547219141,https://stackoverflow.com/questions/54149111/w...,What is the difference between Paypal &quot;pr...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-11,1,"What is the difference between Paypal ""preappr..."
1,54149111,paypal-sandbox,"['paypal', 'paypal-sandbox', 'paypal-subscript...","{'reputation': 4604, 'user_id': 1319542, 'user...",True,606,54165510.0,1,0,1577543072,1547219141,https://stackoverflow.com/questions/54149111/w...,What is the difference between Paypal &quot;pr...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-11,1,"What is the difference between Paypal ""preappr..."
2,54149111,paypal-subscriptions,"['paypal', 'paypal-sandbox', 'paypal-subscript...","{'reputation': 4604, 'user_id': 1319542, 'user...",True,606,54165510.0,1,0,1577543072,1547219141,https://stackoverflow.com/questions/54149111/w...,What is the difference between Paypal &quot;pr...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-11,1,"What is the difference between Paypal ""preappr..."
3,54320426,spring-boot,"['spring-boot', 'paypal', 'access-token', 'res...","{'reputation': 167, 'user_id': 9673158, 'user_...",False,260,NaN,1,0,1577362298,1548220554,https://stackoverflow.com/questions/54320426/h...,How to set paypal get access token param of PO...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-22,0,How to set paypal get access token param of PO...
4,54320426,paypal,"['spring-boot', 'paypal', 'access-token', 'res...","{'reputation': 167, 'user_id': 9673158, 'user_...",False,260,NaN,1,0,1577362298,1548220554,https://stackoverflow.com/questions/54320426/h...,How to set paypal get access token param of PO...,paypal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-22,0,How to set paypal get access token param of PO...


In [36]:
df_tags_questions.to_csv('df_tags_questions'+today_date+'.csv', index=False)